In [1]:
import cv2
import numpy as np

In [2]:
###=============== KNN start =======================
cap = cv2.VideoCapture(0)

In [3]:
face_cascade = cv2.CascadeClassifier("C:/Users/reyna/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_alt.xml")


In [4]:
skip = 0
face_data = []
dataset_path = 'opencv_practice/'
offset = 10

In [5]:
file_name = input("Enter the name of the person :  ")
while True:
    ret,frame = cap.read()
    
    if(ret == False):
        continue
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    #print(faces)
    faces = sorted(faces,key=lambda f:f[2]*f[3])
    
    #pick the last face (largest)
    for face in faces[-1:]:
        x,y,w,h =  face 
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    #extract main face
    face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
    face_section = cv2.resize(face_section,(100,100))
    
    skip += 1
    if(skip%10 == 0):
        face_data.append(face_section)
        print(len(face_data))
         
    cv2.imshow("Cropped",face_section) 
    cv2.imshow("VIDEO FRAME",frame)
        
    keypressed = cv2.waitKey(1) & 0xFF
    if(keypressed == ord('q')):
        break
    if keypressed == ord(' '):
        cv2.imwrite('opencv_practice/cap_face.jpg',frame)

cap.release()
cv2.destroyAllWindows()

Enter the name of the person :  anna


NameError: name 'y' is not defined

In [ ]:
# convert our face list array into a numpy array
face_data = np.array(face_data)
print(face_data.shape)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

In [ ]:
#save this data into file system
np.save(dataset_path + file_name + '.npy',face_data)
print("data successfully saved at " + dataset_path+file_name+'.npy')

In [ ]:
#Building The Face Classifier
# Recognise Faces using the classification algorithm — KNN.
# 1. load the training data (numpy arrays of all the persons)
# x- values are stored in the numpy arrays
# y-values we need to assign for each person
# 2. Read a video stream using opencv
# 3. extract faces out of it
# 4. use knn to find the prediction of face (int)
# 5. map the predicted id to name of the user
# 6. Display the predictions on the screen — bounding box and name

In [25]:
import os

In [26]:
########## KNN CODE ############
def distance(v1, v2):
    # Eucledian
    return np.sqrt(((v1-v2)**2).sum())

In [6]:
def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]
    
    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
########### KNN CODE ##############

In [7]:
#Init Camera
cap = cv2.VideoCapture(0)

In [8]:
# Face Detection
face_cascade = cv2.CascadeClassifier(".conda/envs/tensorflow2/lib/site-packages/cv2/data/haarcascade_frontalface_alt.xml")

In [9]:
skip = 0
dataset_path = 'opencv_practice/'

face_data = []
labels = []

class_id = 0 # Labels for the given file
names = {} #Mapping btw id - name

In [10]:
# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'): #Create a mapping btw class_id and name
        names[class_id] = fx[:-4]
        print('Loaded '+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        #Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

NameError: name 'os' is not defined

In [11]:
# Testing 
while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        #Get the face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        #Predicted Label (out)
        out = knn(trainset,face_section.flatten())

        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
    if key == ord('s'):
        cv2.imwrite('opencv_practice/cap_face_reg.jpg',frame)

cap.release()
cv2.destroyAllWindows()

#==================== KNN End ========================

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [1]:
#================= SVM Start ===================
# pip install sklearn
import os
import pdb
import glob
import scipy.misc 
import argparse
import numpy as np
from sklearn import svm,metrics 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.cross_validation import ShuffleSplit
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
import matplotlib.image as mpimg
import cv2
#import config
import math
import imageio
from skimage.transform import resize

In [2]:
FACE_CASCADE_FILE = ".conda/envs/tensorflow2/lib/site-packages/cv2/data/haarcascade_frontalface_alt.xml"
EYE_CASCADE_FILE = ".conda/envs/tensorflow2/lib/site-packages/cv2/data/haarcascade_eye.xml"
DEFAULT_FACE_SIZE = (200, 200)
RECOGNIZER_OUTPUT_FILE = "opencv_practice\train_result.out"

In [12]:
# convert input image
def convert_to_grey(path):
    files = glob.glob(os.path.join(path,"*/*"))

    for f in files:
        img = cv2.imread(f)
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        str_ing = []
        if(f.find(".jpg")>=0):
            str_ing = f.split(".jpg")
            final = str_ing[0] + ".jpg"
            cv2.imwrite( final, img )
            print (final)
        if(f.find(".png")>=0):
            str_ing = f.split(".png")
            final = str_ing[0] + ".png"
            cv2.imwrite( final, img )
            print (final)
        if(f.find(".jpeg")>=0):
            str_ing = f.split(".jpeg")
            final = str_ing[0] + ".jpeg"
            cv2.imwrite( final, img )
            print( final )
        if(f.find(".pgm")>=0):
            continue
            
convert_to_grey("Fr-svm/orl_faces")
convert_to_grey("Fr-svm/test-data")

NameError: name 'glob' is not defined

In [13]:
#crop_faces
import os.path

cnt = 0
face_cascade = cv2.CascadeClassifier(FACE_CASCADE_FILE)
eye_cascade = cv2.CascadeClassifier(EYE_CASCADE_FILE)

DIR = 'Fr-svm/orl_faces'
numPics = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

pic = 1
files = glob.glob(os.path.join(DIR,"*/*"))


NameError: name 'FACE_CASCADE_FILE' is not defined

In [59]:
for f in files:
    
    img = cv2.imread(f)
    print (f)
    height = img.shape[0]
    width = img.shape[1]
    size = height * width

    if size > (500^2):
        r = 500.0 / img.shape[1]
        dim = (500, int(img.shape[0] * r))
        img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img = img2

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    cas_rejectLevel = 1.3
    cas_levelWeight = 5


    for (x,y,w,h) in faces:
        eyesn = 0
        imgCrop = img[y:y+h,x:x+w]
        #cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            #cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            eyesn = eyesn +1
        if eyesn >= 2:
            #### increase the counter and save 
            cnt +=1
            file_name = f.split("/")[-1] 
            cv2.imwrite(f, imgCrop)

            #cv2.imshow('img',imgCrop)
            print("Image"+str(pic)+" has been processed and cropped")

    pic = pic+1
    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break

#cap.release()
print("All images have been processed!!!")
cv2.destroyAllWindows()
cv2.destroyAllWindows()

Fr-svm/orl_faces\AamairKhan\1.jpeg
Image1 has been processed and cropped
Fr-svm/orl_faces\AamairKhan\10-Reasons-Why-Aamir-Khan-Is-The-Real-Perfectionist-Of-Bollywood-9.jpg
Image2 has been processed and cropped
Fr-svm/orl_faces\AamairKhan\1509611919.jpg
Image3 has been processed and cropped
Fr-svm/orl_faces\AamairKhan\17-Aamir-Khan.jpg
Fr-svm/orl_faces\AamairKhan\533650158.jpg
Fr-svm/orl_faces\AamairKhan\533650216.jpg
Fr-svm/orl_faces\AamairKhan\5a7f2ad5a7e37.image.jpg
Fr-svm/orl_faces\AamairKhan\817460.jpg
Fr-svm/orl_faces\AamairKhan\aamir-khan-2-091214.jpg
Fr-svm/orl_faces\AamairKhan\aamir-khan-20160704165255-2360.jpg
Image10 has been processed and cropped
Fr-svm/orl_faces\AamairKhan\Aamir_Khan_2013.jpg
Image11 has been processed and cropped
Fr-svm/orl_faces\AamairKhan\dangal-aamir-khan-instagram_640x480_51499859868.jpg
Fr-svm/orl_faces\AamairKhan\images (1).jpeg
Fr-svm/orl_faces\AamairKhan\images (2).jpeg
Fr-svm/orl_faces\AamairKhan\images.jpeg
Image15 has been processed and cropped


In [61]:
# multiple faces 
dictionary = {}

for f in files:
    
    img = cv2.imread(f)
    height = img.shape[0]
    width = img.shape[1]
    size = height * width
    temp_img = img
    if size > (500^2):
        r = 500.0 / img.shape[1]
        dim = (500, int(img.shape[0] * r))
        img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img = img2

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    cas_rejectLevel = 1.3
    cas_levelWeight = 5
    pic = 1
    for l in faces:
        #print l
        eyesn = 0
        x = l[0]
        y = l[1]
        w = l[2]
        h = l[3]
        temp_list = (x,y,w,h)
        imgCrop = img[y:y+h,x:x+w]
        cv2.rectangle(temp_img,(x,y),(x+w,y+h),(255,0,0),3)
        cv2.putText(temp_img," face", (x, y), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2, cv2.LINE_AA)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            eyesn = eyesn +1
        if eyesn >= 2:
            #### increase the counter and save 
            file_name = f.split("/")[-1]
            check = file_name.split(".")[0]
            check_formaat = file_name.split(".")[-1]
            ans = "opencv_practice/output/data/" + check + "_"+ str(pic)+"." +check_formaat
            cv2.imwrite(ans, imgCrop)
            temp = "opencv_practice/output/data/" + check +"_"+ str(pic)+ "."+ check_formaat
            pic = pic+1
            temp_dict = {temp:temp_list}
            dictionary.update(temp_dict)
            cv2.imshow('img',temp_img)
            #print("Image"+str(pic)+" has been processed and cropped")
            if cv2.waitKey(200) & 0xFF == ord('q'):
                break
    

#cap.release()
pickle.dump(dictionary, open("opencv_practice/dictionary.txt", 'wb'))
print("All images have been processed!!!")
cv2.destroyAllWindows()
cv2.destroyAllWindows()

All images have been processed!!!


In [62]:
#dictionary
check = {"test30.jpg":"abhishek","test29.jpg":"AamairKhan","test28.jpg":"AamairKhan","test26.jpg":"AkshayKumar","test25.jpg":"AamairKhan","test27.jpg":"AkshayKumar","1.jpg":"siddhartmalhotra","2.jpg":"AliaBhatt","6.jpg":"varundhawan","test7.jpg":"sharukhkhanAliaBhatt","test24.jpg":"AliaBhattsiddhartmalhotra","test29.jpeg":"AkshayKumar","test30.jpeg":"AkshayKumar","test31.jpeg":"AamairKhan","test32.jpg":"AamairKhan","test33.jpeg":"sharukhkhan","test34.jpeg":"sharukhkhan"}


In [13]:
def load_data(path):
    images = []
    labels = []
    files = glob.glob(os.path.join(path,"*/*"))
    #print files
    for f in files:
        #img = scipy.misc.imread(f)
        img = imageio.imread(f)
        bc = resize(img, (112, 92))
        result = bc
        if(np.array(bc).ndim==3):
            result = bc[:,:,0]
        images.append(result)
        #images.append(bc)
        file_name = f.split("/")[-2]
        labels.append(file_name)
    #print(len(images))
    return np.array(images).reshape((len(images),-1)),np.array(labels)


In [93]:
def load_data_test(path):
    images = []
    labels = []
    files = glob.glob(os.path.join(path,"*/*"))
    files.sort()
    for f in files:
        #img = scipy.misc.imread(f)
        img = imageio.imread(f)
        bc = resize(img, (112, 92))
        result = bc[:,:,0]
        images.append(result)
        file_name = f.split("/")[-1]
        labels.append(file_name)
    #print(image.shape)
    return np.array(images).reshape((len(images),-1)),np.array(labels)


In [4]:
#runs for testing only
#========================================================================================


def test_it(model,images,valid_labels,verbose=True):
    prediction = model.predict(images)
    if verbose:
        pretty_print_it(prediction,valid_labels)
    return prediction

def pretty_print_it(prediction,valid_labels):
    assert prediction is not None,"Prediction was not performed"
    print ("==============================================")
    #print metrics.classification_report(valid_labels,prediction)
    print ("==============================================")

def calculate_accuracy(test_labels,prediction,dictionary):
    overall_accuracy = 0.0
    total_img = len(prediction)
    test_path = "opencv_practice/test-data/data/"
    count = 0
    for img in test_labels:
        name = prediction[count]
        count = count + 1
        crop_test_image_path = "opencv_practice/output/data/"+img
        x,y,w,h = dictionary[crop_test_image_path]
        ans = img.split("_")[0]
        ans2 = img.split(".")[-1]
        img =ans + "."+ans2
        if ((check[img] == name) or (check[img].find(name)>=0)) :
            overall_accuracy = overall_accuracy +1
        image_path = test_path  + ans + "."+ans2
        actual_image = cv2.imread(image_path)
        print (image_path)
        cv2.putText(actual_image,name, (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1, cv2.LINE_AA)
        #cv2.putText(actual_image,name,(10, 50),cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 1, cv2.LINE_AA)
        cv2.imshow("camera", actual_image)
        if cv2.waitKey(800) & 0xFF == ord('q'):
            break
    overall_accuracy = (overall_accuracy/total_img)*100.0
    print ("The overall accuracy is :"  +  str(overall_accuracy))

#=======================================================================================


In [14]:
# initialize SVM
gamma=0.001
images,labels = load_data("Fr-svm/orl_faces")
classifier = svm.LinearSVC(verbose=1)
length = images.shape[0]
print ('Loaded images and labels of shape %s and %s' % (images.shape,labels.shape))

Loaded images and labels of shape (116, 10304) and (116,)


In [16]:
#from sklearn.model_selection import train_test_split as tts
# train
def train():
    print ("Starting Training")

    #rs = StratifiedShuffleSplit(length,n_iter=5,test_size=0.25,random_state=None)
    #rs = ShuffleSplit(n_splits=10,test_size=0.25,random_state=0)
    rs = StratifiedShuffleSplit(n_splits=10,test_size=0.25,random_state=0)
    rs.get_n_splits(images,labels)
    #print (rs)
    fold = 1
    #x_train, x_test, income_train, income_test = tts( other_colums, income_column, shuffle = True, stratify = Income_column)
    for train_index,test_index in rs.split(images, labels):
        train_images,train_labels = images[train_index],labels[train_index]
        valid_images,valid_labels = images[test_index],labels[test_index]
        print (train_images)
        svm_classifier = classifier.fit(train_images,train_labels)
        # save the model to disk
        fold+=1

    filename = 'opencv_practice\finalized_model.sav'
    pickle.dump(svm_classifier, open(filename, 'wb'))

In [42]:
#Start SVM
foldername = 'Fr-svm\orl_faces'
fold=5
test_size=0.25
random_state= None
glob_searh='*/*'
verbose=1
model_file= 'opencv_practice\finalized_model.sav'
dictionary={}
if os.path.isfile(model_file):
    print ("model is already present")
    with open("Fr-svm\dictionary.txt","rb") as fp:
        dictionary = pickle.load(fp, encoding='iso-8859-1')
        with open("opencv_practice\finalized_model.sav","rb") as fp:
            model = pickle.load(fp, encoding='iso-8859-1')
            test_images,test_labels = load_data_test("output")
            prediction  = test_it(model,test_images,test_labels)
            print ((prediction))
            print ((test_labels))
            calculate_accuracy(test_labels,prediction,dictionary)
else:
    train()

Starting Training
[[0.52156863 0.5254902  0.49456522 ... 0.36078431 0.36078431 0.36965047]
 [0.10540386 0.09803922 0.09184174 ... 0.07095055 0.0745098  0.0745098 ]
 [0.10813847 0.07852157 0.02297528 ... 0.75733924 0.75268885 0.69769478]
 ...
 [0.02093838 0.02352941 0.03017903 ... 0.10510596 0.11602424 0.24466569]
 [0.07414292 0.02455738 0.0272546  ... 0.50196078 0.49920191 0.49411765]
 [0.47058824 0.46092837 0.44606625 ... 0.96394319 0.95629415 0.74104897]]


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'Fr-svm'

In [14]:
cap.release()
cv2.destroyAllWindows()